# PS2-6: Spam Classification

## Import Library

In [105]:
import collections
from os.path import split

import numpy as np

import Full_Problem_Set.PS2.src.svm as svm
import Full_Problem_Set.PS2.src.util as util

## Load Dataset

In [106]:
#Train
x_train, y_train = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_train.tsv')

#Valid
x_valid, y_valid = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_val.tsv')

#Test
x_test, y_test = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_test.tsv')

## a) Process Spam messages into Numpy array

### 1. Normalization & Splitting

In [107]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    #1. Normalization
    lower_message = message.lower()

    #2. Splitting
    split_message_to_words = lower_message.split()

    return split_message_to_words
    # *** END CODE HERE ***

### 2. Create a dictionary mapping words to integer indices

In [108]:
def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    #1. Convert dataset into array
    messages_arr = np.array(messages)

    #2. Create new dict
    dict_word_indices = {}

    #3. Convert each message in dataset to array
    words_msg_arr = np.empty(len(messages_arr), dtype=object)
    for i in range(len(messages_arr)):
        words_msg_arr[i] = get_words(messages_arr[i])

    #4. Separate words_msg_arr into each word and count the number of it appears
    for i in range(len(words_msg_arr)):
        for j in range(len(words_msg_arr[i])):
            word = words_msg_arr[i][j]
            if word not in dict_word_indices:
                dict_word_indices[word] = 1
            else:
                dict_word_indices[word] += 1

    #5. Delete word in dict that has less than 5 times appear
    for key, value in list(dict_word_indices.items()):
        if value < 5:
            del dict_word_indices[key]

    #6. Convert "the number of word appear" into index (This step is important for retrieve process below)
    cnt = -1
    for key, value in dict_word_indices.items():
        cnt += 1
        dict_word_indices[key] = cnt

    return dict_word_indices
    # *** END CODE HERE ***

### 3. Generate Feature Matrix

In [109]:
def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    #1. Convert each message into word
    word_msg_arr = [None] * len(messages)
    for i in range(len(messages)):
        word_msg_arr[i] = get_words(messages[i])

    #.2 Create matrix feature (row - message[i]; column - dict of word; each entry is the times of word(in dict) appear in each message[i])
    feature_matrix = np.zeros((len(messages), len(word_dictionary)))

    for i in range(len(word_msg_arr)):
        for j in range(len(word_msg_arr[i])):
            if word_msg_arr[i][j] in word_dictionary:
                key = word_msg_arr[i][j]
                value = word_dictionary[key]
                feature_matrix[i][value] += 1

    return feature_matrix
    # *** END CODE HERE ***

## b) Implement Naive Bayes classifier for spam classification with Multinomial Event Model

### 1. Fit Naive Bayes Model

In [110]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    #1. Get feature_matrix shape
    m, n = matrix.shape

    #-----------------------------------------------------
    #2. Calculate Phi_y
    #phi_y1
    phi_y1 = 0
    for i in range(m):
        if labels[i] == 1:
            phi_y1 += 1 / m * 1

    #phi_y0
    phi_y0 = 0
    for i in range(m):
        if labels[i] == 0:
            phi_y0 += 1/ m
    #-----------------------------------------------------
    #3. Calculate Phi_k_y=1
    #numerator: Count number of k appear in class y = 1
    k_appear_cnt_y1 = np.zeros(n)
    for i in range(m):
        if labels[i] == 1:
            k_appear_cnt_y1 += matrix[i]

    #denominator: Count number of word in class y = 1
    #METHOD 1: COUNT ALL THE NUMBER OF EACH CLASS Y = 1
    # total_word_y1 = 0
    # for i in range(m):
    #     if labels[i] == 1:
    #         for j in range(n):
    #             total_word_y1 += matrix[i][j]

    #METHOD 2: CLASS 1 JUST CONTAIN ALL WORD IN k_appear_cnt_y1
    total_word_y1 = 0
    for j in range(n):
        total_word_y1 += k_appear_cnt_y1[j]

    #Apply Laplace Smoothing
    phi_k_y1 = (1 + k_appear_cnt_y1) / (total_word_y1 + n)
    #-----------------------------------------------------
    #4. Calculate Phi_k_y=0
    #numerator: Count number of k appear in class y = 0
    k_appear_cnt_y0 = np.zeros(n)
    for i in range(m):
        if labels[i] == 0:
            k_appear_cnt_y0 += matrix[i]

    #denominator: Count number of word in class y = 0
    total_word_y0 = 0
    for j in range(n):
        total_word_y0 += k_appear_cnt_y0[j]

    #Apply Laplace Smoothing
    phi_k_y0 = (1 + k_appear_cnt_y0) / (total_word_y0 + n)
    #-----------------------------------------------------

    return phi_y1, phi_y0, phi_k_y1, phi_k_y0
    # *** END CODE HERE ***

### 2. Predict Naive Bayes Model

In [111]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containing the predictions from the model
    """
    # *** START CODE HERE ***
    #1. Parameter
    phi_y1, phi_y0, phi_k_y1, phi_k_y0 = model

    #2. Predict phase (Compare P(y=1|x) & P(y=0|x))
    #METHOD 1: Do not using Matrix
    # m, n = matrix.shape
    # sum_log = np.zeros(m)
    # for i in range(m):
    #     for j in range(n):
    #         sum_log[i] += matrix[i][j] * np.log(phi_k_y1[j] / phi_k_y0[j])
    # return sum_log + np.log(phi_y1 / phi_y0) >= 0

    #METHOD 2: Using Matrix
    return matrix @ np.log(phi_k_y1 / phi_k_y0) + np.log(phi_y1 / phi_y0) >= 0
    # *** END CODE HERE ***

## c) Obtain 5 most indicative tokens in SPAM Class

In [112]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    #1. Parameter
    _, _, phi_k_y1, phi_k_y0 = model

    #2. Compare P(xj=k|y=1) & P(xj=k|y=0)
    #Get dictionary length
    n = len(dictionary)

    #Calculate ratio between y = 1 class and y = 0 class
    log_ratio = np.zeros(n)
    for j in range(n):
        log_ratio[j] = np.log(phi_k_y1[j] / phi_k_y0[j])

    #Determine the word that has higher probability in class y = 1
    #Create new array to save word in class 1
    prob_k_y1 = np.zeros(n)
    for j in range(n):
        if log_ratio[j] >= 0:
            prob_k_y1[j] = log_ratio[j]

    #Create new dictionary that contain all words in class y = 1
    y1_dict = dictionary.copy()
    for key, value in y1_dict.items():
        y1_dict[key] = prob_k_y1[value]

    #Create key of y1_word (Contain Word in class y = 1)
    word_y1 = []
    #Create value of y1_word (Contain Probability which response to Word)
    prob_y1 = []
    for key, value in y1_dict.items():
        word_y1.append(key)
        prob_y1.append(value)

    #Determine top 5 word that has the most probability in class y = 1
    #Selection sort
    for j in range(n):
        idx_max = j
        max_prob = prob_y1[j]
        most_word = word_y1[j]

        #Find max
        for k in range(j+1, n):
            if prob_y1[k] > max_prob:
                idx_max = k
                max_prob = prob_y1[k]
                most_word = word_y1[k]

        #Swap prob
        temp_prob = prob_y1[j]
        prob_y1[j] = max_prob
        prob_y1[idx_max] = temp_prob
        #Swap word
        temp_word = word_y1[j]
        word_y1[j] = most_word
        word_y1[idx_max] = temp_word

    #Top 5 words appear the most in class y = 1
    top_5_word_y1 = []
    for j in range(5):
        top_5_word_y1.append(word_y1[j])

    return top_5_word_y1
    # *** END CODE HERE ***

## d)

In [114]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider

    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    #1. Train and predict SVM & Calculate accuracy
    best_accuracy = 0
    best_radius = 0
    for radius in radius_to_consider:
        #Train and Pred
        svm_train_and_pred = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)

        # #Accuracy
        m, _ = val_matrix.shape
        mse = 0
        for i in range(m):
            if svm_train_and_pred[i] == val_labels[i]:
                mse += 1 / m

        if mse > best_accuracy:
            best_accuracy = mse
            best_radius = radius

    return best_radius
    # *** END CODE HERE ***

## Main

In [115]:
train_messages, train_labels = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_train.tsv')
val_messages, val_labels = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_val.tsv')
test_messages, test_labels = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_test.tsv')

dictionary = create_dictionary(train_messages)

util.write_json('./output/p06_dictionary', dictionary)

train_matrix = transform_text(train_messages, dictionary)

np.savetxt('./output/p06_sample_train_matrix', train_matrix[:100,:])

val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

np.savetxt('./output/p06_naive_bayes_predictions', naive_bayes_predictions)

naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

util.write_json('./output/p06_top_indicative_words', top_5_words)

optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])

util.write_json('./output/p06_optimal_radius', optimal_radius)

print('The optimal SVM radius was {}'.format(optimal_radius))

svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)

svm_accuracy = np.mean(svm_predictions == test_labels)

print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))


Naive Bayes had an accuracy of 0.978494623655914 on the testing set
The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']
The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9731182795698925 on the testing set
